# CorALS - Quickstart

## Prepare parallelization

Before running anything, we make sure that `numpy` will not  oversubscribe CPUs and slow things down.
Note that this has to be executed **before importing `numpy`**.

* For full correlation matrix calculation, setting `n_threads > 1` can be used to parallelize the calculation.
* For the top-k approaches, setting `n_threads=1` makes the most sense, since parallelization is specified separately.

In [2]:
import pandas as pd
from corals.threads import set_threads_for_external_libraries
set_threads_for_external_libraries(n_threads=1)



/opt/homebrew/lib/python3.10/site-packages/corals/threads.py:48: UserWarning: This function should be called before `numpy` or similar modules are imported.
  warnings.warn("This function should be called before `numpy` or similar modules are imported.")


In [44]:
# Load the sample-gene matrix data
sample_gene_matrix = pd.read_csv('crc_pancan_mrna.csv', index_col=0)

In [53]:
sig_name = "Primary (Early Age Diagnosis)"
# Load the sample groups
sample_groups = pd.read_csv(f'crc_eas_signatures/spl-full_CRCEAS_{sig_name}.csv', index_col=0)
sample_groups = sample_groups.rename(columns={'cluster': 'group'})

# Load the gene groups
gene_groups = pd.read_csv(f'crc_eas_signatures/sig_CRCEAS_{sig_name}.csv', index_col=0)
gene_groups = gene_groups.rename(columns={'cluster': 'group'})

 

In [ ]:
# Iterate over each group of samples and genes
for group in sample_groups['group'].unique():
    group_samples = sample_groups[sample_groups['group'] == group].index
    group_genes = gene_groups[gene_groups['group'] == group].index
    # Get the intersection of available samples and genes
    available_samples = group_samples.intersection(sample_gene_matrix.columns)
    available_genes = group_genes.intersection(sample_gene_matrix.index)
    # Extract the corresponding subset of the sample-gene matrix data
    group_data = sample_gene_matrix.loc[available_genes, available_samples].transpose()
    group_data = group_data.fillna(0)

    # df to array
    X = group_data.to_numpy()
    # reusing correlation from the top-k example
    # runtime: ~5 sec with `n_jobs=8`


    # FULL
    from corals.correlation.full.default import cor_full
    cor_values = cor_full(X)
    cor_full = pd.DataFrame(cor_values, columns = group_data.columns)
    cor_full.index = group_data.columns
    # Save the extracted data to a new CSV file
    filename = f"corfull_{sig_name}_{group}.csv"
    cor_full.to_csv(filename)

    # TOP K

    n_samples = X.shape[0]
    n_features = X.shape[1]
    print(n_samples)
    print(n_features)

    from corals.correlation.topk.default import cor_topk
    cor_topk_values, cor_topk_coo = cor_topk(X, correlation_type="spearman", k=0.01, n_jobs=8)

    from corals.correlation.utils import derive_pvalues, multiple_test_correction
    # calculate p-values
    pvalues = derive_pvalues(cor_topk_values, n_samples)



    cor = pd.DataFrame(cor_topk_values, columns=['cor'])
    p = pd.DataFrame(pvalues, columns=['pvalue'])
    coo = pd.DataFrame(cor_topk_coo).transpose()
    pair = pd.DataFrame({
        'start': group_data.columns[cor_topk_coo[0]],
        'end': group_data.columns[cor_topk_coo[1]]
    })
    # concatenate the two dataframes horizontally
    df_concat = pd.concat([coo, pair, cor, p], axis=1)
    # Save the extracted data to a new CSV file


    # Save the extracted data to a new CSV file
    filename = f"topk_{sig_name}_{group}.csv"
    df_concat.to_csv(filename)


In [85]:
group = "G1"
group_samples = sample_groups[sample_groups['group'] == group].index
group_genes = gene_groups[gene_groups['group'] == group].index
# Get the intersection of available samples and genes
available_samples = group_samples.intersection(sample_gene_matrix.columns)
available_genes = group_genes.intersection(sample_gene_matrix.index)
# Extract the corresponding subset of the sample-gene matrix data
group_data = sample_gene_matrix.loc[available_genes, available_samples].transpose()
group_data = group_data.fillna(0)


# df to array
X = group_data.to_numpy()
# reusing correlation from the top-k example
# runtime: ~5 sec with `n_jobs=8`


# FULL
from corals.correlation.full.default import cor_full
cor_values = cor_full(X)
cor_full = pd.DataFrame(cor_values, columns = group_data.columns)
cor_full.index = group_data.columns
# Save the extracted data to a new CSV file
filename = f"corfull_{sig_name}_{group}.csv"
cor_full.to_csv(filename)

# TOP K
n_samples = X.shape[0]
n_features = X.shape[1]
print(n_samples)
print(n_features)

from corals.correlation.topk.default import cor_topk
cor_topk_values, cor_topk_coo = cor_topk(X, correlation_type="spearman", k=0.1, n_jobs=8)

from corals.correlation.utils import derive_pvalues, multiple_test_correction
# calculate p-values
pvalues = derive_pvalues(cor_topk_values, n_samples)



cor = pd.DataFrame(cor_topk_values, columns=['cor'])
p = pd.DataFrame(pvalues, columns=['pvalue'])
coo = pd.DataFrame(cor_topk_coo).transpose()
pair = pd.DataFrame({
    'start': group_data.columns[cor_topk_coo[0]],
    'end': group_data.columns[cor_topk_coo[1]]
})
# concatenate the two dataframes horizontally
df_concat = pd.concat([coo, pair, cor, p], axis=1)
# Save the extracted data to a new CSV file


# Save the extracted data to a new CSV file
filename = f"topk_{sig_name}_{group}.csv"
df_concat.to_csv(filename)






5
255


/opt/homebrew/lib/python3.10/site-packages/corals/correlation/full/matmul.py:53: RuntimeWarning: invalid value encountered in divide
  XX /= std * X.shape[0]
/opt/homebrew/lib/python3.10/site-packages/corals/correlation/full/matmul.py:59: RuntimeWarning: invalid value encountered in divide
  YY = X / std
/opt/homebrew/lib/python3.10/site-packages/corals/correlation/utils.py:8: RuntimeWarning: invalid value encountered in divide
  Xh /= np.std(Xh, axis=0) * np.sqrt(X.shape[0])


ValueError: Input contains NaN.